In [116]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('C:/Users/timei/Documents/Datalogi/Semester6/Bachelor/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\timei\Documents\Datalogi\Semester6\Bachelor\pm4py-dcr
C:\Users\timei\Documents\Datalogi\Semester6\Bachelor\pm4py-dcr


In [117]:
import pm4py
from copy import deepcopy

In [118]:
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.discovery.dcr_discover import algorithm as dcr_alg

In [209]:
dcr = {
    'events': {'release', 'archive', 'delete', 'readmit', 'unarchive'},
    'labels': set(),
    'labelMapping': set(),
    'conditionsFor': {'delete': {'release'}, 'unarchive':{'release'}, 'archive':{'release'}},  # this should be a dict with events as keys and sets as values
    'milestonesFor': {'delete': {'archive'}},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'release':{'archive', 'delete', 'unarchive'}},
    'excludesTo': {'readmit':{'unarchive', 'archive', 'delete'}},
    'conditionsForDelays': {'unarchive':{('archive', 2920)}},  # this should be a dict with events as keys and tuples as values
    'responseToDeadlines': {'release':{('archive',14), ('delete',14)}},
    'marking': {'executed': set(),
                'included': {'release', 'archive', 'delete', 'readmit', 'unarchive'},
                'pending': set(),
                'executedTime': {}, # Gives the time since a event was executed
                'pendingDeadline': {} # The deadline until an event must be executed 
                }
}

In [210]:
dcr_semantics.enabled(dcr)

{'readmit', 'release'}

In [211]:
dcr_semantics.is_accepting(dcr)

True

In [212]:
event_to_execute = 'release' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

{'release', 'archive', 'readmit'}


In [193]:
event_to_execute = 'hallo' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event

False

In [139]:
test = {'hej': 14, 'hello':13}
print(test)
test["hi"] = (25)
print(test)
test['hi'] = 10
print(test)

{'hej': 14, 'hello': 13}
{'hej': 14, 'hello': 13, 'hi': 25}
{'hej': 14, 'hello': 13, 'hi': 10}


In [134]:
real = {('hej', 1), ('hello', 2), ('whatup', 3)}
for (e_prime, k) in real:
    print(e_prime)
    print(k)

hej
1
hello
2
whatup
3


In [213]:
def time_step(time, dcr): # pendingDeadline can be 0 when excluded
    deadline = dcr_semantics.find_next_deadline(dcr)
    if deadline == None or deadline - time >= 1:
        for e in dcr['marking']['pendingDeadline']:
            dcr['marking']['pendingDeadline'][e] =  max(dcr['marking']['pendingDeadline'][e] - time, 0)
        for e in dcr['conditionsForDelays']:
            for (e_prime, k) in dcr['conditionsForDelays'][e]:
                if e_prime in dcr['marking']['executed']:
                    dcr['marking']['executedTime'][e_prime] = min(dcr['marking']['executedTime'][e_prime] + time, max_executed_time(e_prime, dcr))
    else:
        print('unlucko')

In [216]:
print(time_step(13, dcr))

None


In [217]:
dcr_semantics.find_next_deadline(dcr)

1

In [173]:
def max_executed_time(event, dcr):
    max_delay = None
    for e in dcr['conditionsForDelays']:
        for (e_prime, k) in dcr['conditionsForDelays'][e]:
            if e_prime in dcr['marking']['executed'] and e_prime == event:
                if max_delay == None or k > max_delay:
                    max_delay = k
    return max_delay

In [174]:
print(max_executed_time("archive", dcr))

2920
